<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# Redes Neuronales Convulucionales (CNN) multivariadas para series de tiempo
## *Modelos no lineales para pronósitico*  - Pedro Martinez

---

# CNN 1D multivariadas para pronóstico del clima

Hoy en día, Google y OpenWeather usan variantes de CNN en sus pipelines de nowcasting meteorológico ya que son capaces de detectar patrones temporales locales en series de tiempo, como picos de temperatura o tendencias estacionales.

Usaremos datos reales de la API [Open-Meteo](https://open-meteo.com/) para **predecir la temperatura de las próximas 48 h**, empleando las variables:

| Variable | Descripción | Unidad | Fuente |
|-----------|--------------|---------|---------|
| **Temperatura** | Temperatura del aire a 2 metros | °C (grados Celsius) | `temperature_2m` |
| **Humedad relativa** | Porcentaje de saturación de vapor de agua | % | `relative_humidity_2m` |
| **Nubosidad** | Cobertura nubosa promedio | % | `cloud_cover` |
| **Velocidad del viento** | Velocidad del viento a 10 metros | m/s | `wind_speed_10m` |

---

Una CNN aplicada a series de tiempo multivariables procesa una secuencia de observaciones:

$$
X_t =
\begin{bmatrix}
x_{t,1} & x_{t,2} & \dots & x_{t,n}
\end{bmatrix}
\in \mathbb{R}^{n}
$$

donde $n$ es el número de variables de entrada (en este caso, 4).

Si utilizamos una **ventana temporal de tamaño $T$**, la entrada al modelo es:

$$
\mathbf{X} =
\begin{bmatrix}
X_{t-T+1} \\
X_{t-T+2} \\
\vdots \\
X_t
\end{bmatrix}
\in \mathbb{R}^{T \times n}
$$

Cada capa convolucional aplica un conjunto de filtros (*kernels*) que aprenden patrones espacio-temporales en la ventana:

$$
h_j^{(l)}(t) = f \!\left(
\sum_{i=1}^{n}
\sum_{k=0}^{K-1}
w_{ij}^{(l)}(k) \, x_i(t - k)
+ b_j^{(l)}
\right)
$$

donde:

- $w_{ij}^{(l)}(k)$ son los pesos del filtro $j$ en la posición $k$ de la capa $l$.  
- $b_j^{(l)}$ es el sesgo asociado al filtro $j$.  
- $f(\cdot)$ es una función de activación no lineal (típicamente **ReLU**).  
- $K$ es el tamaño del kernel (por ejemplo, 3 horas).

El resultado de esta capa convolucional se **aplasta (*flatten*)** y se conecta a una capa **densa (fully connected)** para generar la predicción final.

---

## Interpretación

- La CNN aprende **patrones locales** (por ejemplo, cómo la humedad y la nubosidad afectan la temperatura en las últimas horas).  
- Al combinar varias variables de entrada, la red actúa como un **modelo multivariable no lineal**.  
- El entrenamiento minimiza una función de pérdida (MSE):


In [117]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import requests
import pandas as pd
import plotly.express as px
import tensorflow as tf
from tensorflow.keras import layers, models, Input
import plotly.graph_objects as go
from datetime import datetime, timedelta, timezone
import pytz


In [118]:
CITY_TZ = "America/Mexico_City"
lat, lon = 20.31, -103.18  # Chapala

tz = pytz.timezone(CITY_TZ)

# --- API Open-Meteo ---
url = (
    "https://api.open-meteo.com/v1/forecast"
    f"?latitude={lat}&longitude={lon}"
    "&hourly=temperature_2m,relative_humidity_2m,cloud_cover,wind_speed_10m"
    "&past_days=15"
    f"&timezone={CITY_TZ}"
)
res = requests.get(url).json()

# --- Construcción del DataFrame ---
df = pd.DataFrame({
    "time": pd.to_datetime(res["hourly"]["time"]),
    "temp": res["hourly"]["temperature_2m"],
    "humidity": res["hourly"]["relative_humidity_2m"],
    "clouds": res["hourly"]["cloud_cover"],
    "wind": res["hourly"]["wind_speed_10m"],
}).set_index("time")

# 🔹 Asegurar zona local
df.index = df.index.tz_localize(tz)

# 🔹 Calcular hora local actual y filtrar HASTA una hora atrás
now_local = datetime.now(tz)
cutoff = now_local - timedelta(hours=1)
df = df[df.index <= cutoff]


In [119]:
fig = px.line(df, y=["temp","humidity","clouds","wind"],
              title="Variables meteorológicas",
              template="plotly_white")
fig.show()

En equipos de 2 personas, realizar las siguientes partes:

Seleccionar variable objetivo

In [120]:
TARGET = "humidity"
FEATURES = ["temp", "clouds", "wind"]

# Mantén solo columnas necesarias y descarta filas con NA
df_ml = df[[TARGET] + FEATURES].dropna().copy()

print(df_ml.tail(3))
print("Shape:", df_ml.shape)

                           humidity  temp  clouds  wind
time                                                   
2025-10-31 16:00:00-06:00        32  25.5      27   1.1
2025-10-31 17:00:00-06:00        34  24.9      65   0.5
2025-10-31 18:00:00-06:00        41  23.6      56   3.3
Shape: (379, 4)


Escalar los datos

In [121]:
from sklearn.preprocessing import MinMaxScaler

# Preparar scalers (se ajustan después del split)
x_scaler = MinMaxScaler(feature_range=(0, 1))
y_scaler = MinMaxScaler(feature_range=(0, 1))

print("Scalers listos. Se ajustarán con TRAIN en el siguiente paso.")


Scalers listos. Se ajustarán con TRAIN en el siguiente paso.


Crear ventanas temporales


*   Se sugiere utilizar 12 o 24 valores anteriores


In [122]:
import numpy as np

def make_windows(X: np.ndarray,
                 y: np.ndarray,
                 window_size: int = 24,
                 horizon: int = 48,
                 step: int = 1):
    """
    Genera pares (ventana, objetivo) para multi-step forecasting.
    - X: array (T, n_features)
    - y: array (T,) o (T, 1)
    - window_size: # de pasos históricos por muestra (p.ej., 24 horas)
    - horizon: # de pasos futuros a predecir (p.ej., 48 horas)
    - step: salto entre ventanas consecutivas

    Return:
      Xw: (num_samples, window_size, n_features)
      Yw: (num_samples, horizon)  # vector multi-step
      idx: índices iniciales (timestamps opcionales si llevas tracking)
    """
    if y.ndim == 2 and y.shape[1] == 1:
        y = y.ravel()

    T = len(y)
    samples = []
    targets = []
    starts = []

    end_limit = T - horizon
    start = window_size

    for i in range(start, end_limit + 1, step):
        x_win = X[i - window_size:i, :]                 # (window_size, n_features)
        y_out = y[i:i + horizon]                        # (horizon,)
        samples.append(x_win)
        targets.append(y_out)
        starts.append(i - 1)  # índice del último punto observado

    if len(samples) == 0:
        return np.empty((0, window_size, X.shape[1])), np.empty((0, horizon)), np.array([])

    Xw = np.stack(samples, axis=0)
    Yw = np.stack(targets, axis=0)
    idx = np.array(starts)

    return Xw, Yw, idx

print("Función make_windows definida (aún no se usa).")


Función make_windows definida (aún no se usa).


Dividir en entrenamiento y prueba

In [123]:
import pandas as pd

# --------------------------------
# Sugerencia de split: últimas 3 d
# --------------------------------
TEST_DAYS = 3
split_time = df_ml.index.max() - pd.Timedelta(days=TEST_DAYS)

df_train = df_ml.loc[:split_time].copy()
df_test  = df_ml.loc[split_time + pd.Timedelta(hours=1):].copy()

print("Rango TRAIN:", df_train.index.min(), "→", df_train.index.max(), "| rows:", len(df_train))
print("Rango TEST :", df_test.index.min(),  "→", df_test.index.max(),  "| rows:", len(df_test))

# -------------------------------
# Ajustar scalers SOLO con TRAIN
# -------------------------------
X_train_raw = df_train[FEATURES].to_numpy(dtype=float)
y_train_raw = df_train[[TARGET]].to_numpy(dtype=float)   # 2D para scaler

x_scaler.fit(X_train_raw)
y_scaler.fit(y_train_raw)

# Transformar TRAIN/TEST
X_train = x_scaler.transform(X_train_raw)
y_train = y_scaler.transform(y_train_raw).ravel()        # 1D para comodidad

X_test  = x_scaler.transform(df_test[FEATURES].to_numpy(dtype=float))
y_test  = y_scaler.transform(df_test[[TARGET]].to_numpy(dtype=float)).ravel()

print("Shapes (X_train, y_train):", X_train.shape, y_train.shape)
print("Shapes (X_test , y_test ):", X_test.shape,  y_test.shape)

# Parámetros por defecto sugeridos para luego hacer ventanas
WINDOW_SIZE = 24   # 24 horas de historia
HORIZON     = 48   # predecir 48 horas a futuro


Rango TRAIN: 2025-10-16 00:00:00-06:00 → 2025-10-28 18:00:00-06:00 | rows: 307
Rango TEST : 2025-10-28 19:00:00-06:00 → 2025-10-31 18:00:00-06:00 | rows: 72
Shapes (X_train, y_train): (307, 3) (307,)
Shapes (X_test , y_test ): (72, 3) (72,)


Construir la CNN en Keras
- Recordando las recomendaciones de la clase pasada
- Cambiar número de filtros (16, 32, 64)
- Cambiar tamaño del kernel (2, 3, 5)
- Cambiar función de activación (relu, tanh)
- Cambiar batch_size o epochs

In [124]:
Xw_train, Yw_train, _ = make_windows(X_train, y_train, WINDOW_SIZE, HORIZON)
Xw_test,  Yw_test,  _ = make_windows(X_test,  y_test,  WINDOW_SIZE, HORIZON)

In [125]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras import regularizers
import numpy as np

# Reproducibilidad
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

n_features = Xw_train.shape[2]

model = models.Sequential([
    Input(shape=(WINDOW_SIZE, n_features)),

    # Stem
    layers.Conv1D(64, 5, padding='causal', activation='gelu',
                  kernel_regularizer=regularizers.l2(3e-4)),
    layers.LayerNormalization(),
    layers.Dropout(0.1),

    layers.Conv1D(64, 5, padding='causal', dilation_rate=2, activation='gelu',
                  kernel_regularizer=regularizers.l2(3e-4)),
    layers.LayerNormalization(),
    layers.Dropout(0.15),

    layers.Conv1D(64, 5, padding='causal', dilation_rate=4, activation='gelu',
                  kernel_regularizer=regularizers.l2(3e-4)),
    layers.LayerNormalization(),
    layers.Dropout(0.15),

    layers.Conv1D(64, 5, padding='causal', dilation_rate=8, activation='gelu',
                  kernel_regularizer=regularizers.l2(3e-4)),
    layers.LayerNormalization(),
    layers.Dropout(0.15),

    layers.Conv1D(128, 1, padding='causal', activation='gelu',
                  kernel_regularizer=regularizers.l2(3e-4)),
    layers.LayerNormalization(),

    layers.Lambda(lambda t: t[:, -1, :]),

    # Cabeza densa
    layers.Dense(128, activation='gelu',
                 kernel_regularizer=regularizers.l2(3e-4)),
    # layers.Dropout(0.2),
    layers.Dense(64, activation='gelu',
                 kernel_regularizer=regularizers.l2(3e-4)),

    layers.Dense(HORIZON)  # lineal (sin regularización aquí es OK)
])


model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.Huber(delta=1.0),
    metrics=[tf.keras.metrics.MAE]
)

model.summary()


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_46 (Conv1D)              │ (None, 24, 64)         │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_45          │ (None, 24, 64)         │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_47 (Conv1D)              │ (None, 24, 64)         │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_46          │ (None, 24, 64)         │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_48 (Conv1D)              │ (None, 24, 64)         │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_47          │ (None, 24, 64)         │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_49 (Conv1D)              │ (None, 24, 64)         │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_48          │ (None, 24, 64)         │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_42 (Dropout)            │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_50 (Conv1D)              │ (None, 24, 128)        │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_49          │ (None, 24, 128)        │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_8 (Lambda)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 48)             │         3,120 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,632 (389.19 KB)

 Trainable params: 99,632 (389.19 KB)

 Non-trainable params: 0 (0.00 B)

Entrenar el modelo utilizando una semilla para reproducibilidad

In [126]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5, verbose=1),
]

history = model.fit(
    Xw_train, Yw_train,
    epochs=100,
    batch_size=64,
    validation_split=0.2,   # Keras usa el tramo final cuando shuffle=False
    shuffle=False,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/100


3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 224ms/step - loss: 0.4332 - mean_absolute_error: 0.6328 - val_loss: 0.3017 - val_mean_absolute_error: 0.4486 - learning_rate: 0.0010
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.2872 - mean_absolute_error: 0.4302 - val_loss: 0.2556 - val_mean_absolute_error: 0.3825 - learning_rate: 0.0010
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.2478 - mean_absolute_error: 0.3635 - val_loss: 0.2347 - val_mean_absolute_error: 0.3482 - learning_rate: 0.0010
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.2249 - mean_absolute_error: 0.3185 - val_loss: 0.2176 - val_mean_absolute_error: 0.3184 - learning_rate: 0.0010
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.2079 - mean_absolute_error: 0.2836 - val_loss: 0.2046 - val_mean_absolute_error: 0.2980 - learning_rate: 0.0010
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.1941 - mean_absolute_error: 0.2554 - val_loss: 0.1940 - val_mean_absolute_error: 0.2813 - l

Evaluar y visualizar utilizando plotly

In [127]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Predicción sobre todas las ventanas de TEST
Yhat_test = model.predict(Xw_test, verbose=0)

# Invertir escala a unidades reales (% humedad)
Yhat_test_inv = y_scaler.inverse_transform(Yhat_test.reshape(-1, 1)).reshape(Yhat_test.shape)
Y_test_inv    = y_scaler.inverse_transform(Yw_test.reshape(-1, 1)).reshape(Yw_test.shape)

# Métricas agregadas (aplanando todo el horizonte y todas las ventanas)
mae  = mean_absolute_error(Y_test_inv.ravel(), Yhat_test_inv.ravel())
rmse = mean_squared_error(Y_test_inv.ravel(), Yhat_test_inv.ravel())

# MAPE robusto (evita división por cero)
mape = (np.abs((Y_test_inv - Yhat_test_inv) / np.clip(np.abs(Y_test_inv), 1e-6, None))).mean() * 100

print(f"TEST | MAE: {mae:0.3f}  RMSE: {rmse:0.3f}  MAPE: {mape:0.2f}%")


TEST | MAE: 10.534  RMSE: 155.874  MAPE: 23.40%


In [128]:
import pandas as pd
import plotly.express as px

hist_df = pd.DataFrame({
    "epoch": np.arange(len(history.history["loss"])),
    "loss": history.history["loss"],
    "val_loss": history.history["val_loss"]
})
fig_hist = px.line(hist_df, x="epoch", y=["loss", "val_loss"], title="Histórico de entrenamiento (MSE)")
fig_hist.update_layout(yaxis_title="MSE")
fig_hist.show()


In [129]:
import plotly.graph_objects as go

# Volvemos a construir idx para TEST (posiciones temporales dentro de df_test)
_, _, idx_test = make_windows(X_test, y_test, WINDOW_SIZE, HORIZON)

# Timestamps de TEST (ya están en tz local por tu preproc)
times_test = df_test.index

# Elegimos la ÚLTIMA ventana de test para visualizar
i = -1
start_pos = idx_test[i]
forecast_times = times_test[start_pos+1 : start_pos+1+HORIZON]  # próximos HORIZON puntos

y_true_line = Y_test_inv[i]
y_pred_line = Yhat_test_inv[i]

fig_fc = go.Figure()
fig_fc.add_scatter(x=forecast_times, y=y_true_line, mode="lines+markers", name="Real (humedad)")
fig_fc.add_scatter(x=forecast_times, y=y_pred_line, mode="lines+markers", name="Predicción (CNN)")
fig_fc.update_layout(
    title=f"Pronóstico {HORIZON} h — humedad relativa (%)",
    xaxis_title="Tiempo",
    yaxis_title="Humedad (%)",
    template="plotly_white"
)
fig_fc.show()


In [130]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# --- 1) Serie completa (verdadera) ---
# df_ml tiene TARGET en unidades reales; train/test ya están particionados
full_truth = df_ml[TARGET]                      # (toda la historia)
train_truth = df_train[TARGET]
test_truth  = df_test[TARGET]

# --- 2) Reconstruir timestamps por ventana de TEST y promediar predicciones solapadas ---
# Recalcular índices de inicio por ventana en TEST (coherentes con tus X_test/y_test)
_, _, idx_test = make_windows(X_test, y_test, WINDOW_SIZE, HORIZON)
times_test = df_test.index

# Asegúrate de tener Yhat_test (en escala invertida, % humedad)
# Yhat_test_inv: shape = (num_windows, HORIZON)

agg = {}  # timestamp -> [sum_pred, count]
num_windows = Yhat_test_inv.shape[0]

for j in range(num_windows):
    start_pos = int(idx_test[j])
    # rango de timestamps para el horizonte de esta ventana
    t_slice = times_test[start_pos + 1 : start_pos + 1 + HORIZON]
    y_pred_slice = Yhat_test_inv[j][:len(t_slice)]

    for ts, val in zip(t_slice, y_pred_slice):
        if ts not in agg:
            agg[ts] = [float(val), 1]
        else:
            agg[ts][0] += float(val)
            agg[ts][1] += 1

# Promedio por timestamp (para suavizar solapes)
pred_series = (
    pd.Series({ts: s/c for ts, (s, c) in agg.items()})
    .sort_index()
)
pred_series.name = "Predicción (CNN)"

# --- 3) Gráfica unificada: historia completa + segmentación train/test + predicción agregada ---
fig = go.Figure()

# Historia completa (verdad)
fig.add_trace(go.Scatter(
    x=full_truth.index, y=full_truth.values,
    name="Humedad (real) – completo", mode="lines"
))

# Overlay para resaltar TEST
fig.add_vrect(
    x0=test_truth.index.min(), x1=test_truth.index.max(),
    fillcolor="lightgray", opacity=0.25, line_width=0, layer="below",
)

# Truth solo en TEST (opcional, para contraste más visible)
fig.add_trace(go.Scatter(
    x=test_truth.index, y=test_truth.values,
    name="Humedad (real) – TEST", mode="lines"
))

# Predicción agregada sobre TEST
fig.add_trace(go.Scatter(
    x=pred_series.index, y=pred_series.values,
    name=f"Predicción CNN (h={HORIZON})", mode="lines+markers"
))

fig.update_layout(
    title=f"Serie completa, segmento TEST y predicción agregada (H={HORIZON})",
    xaxis_title="Tiempo",
    yaxis_title="Humedad (%)",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()


In [131]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# 1) Última ventana real (features) y predicción a futuro
last_ts = df_ml.index[-1]
window_df = df_ml[FEATURES].iloc[-WINDOW_SIZE:]                # (WINDOW_SIZE, n_features)
X_last = x_scaler.transform(window_df.to_numpy(dtype=float))   # escala con el scaler ya ajustado
X_last = X_last.reshape(1, WINDOW_SIZE, len(FEATURES))         # (1, W, n_features)

y_future_scaled = model.predict(X_last, verbose=0).reshape(-1, 1)           # (HORIZON, 1) en escala [0,1]
y_future = y_scaler.inverse_transform(y_future_scaled).ravel()              # humedad en unidades reales

# 2) Construir eje temporal futuro (1 hora después del último timestamp, HORIZON puntos)
future_index = pd.date_range(
    start=last_ts + pd.Timedelta(hours=1),
    periods=HORIZON,
    freq="H",
    tz=last_ts.tz if hasattr(last_ts, "tzinfo") else None
)

# 3) (Opcional) Muestra un tramo reciente de la verdad para contexto
HISTORY_HOURS = 72  # 3 días de historia
hist_start = max(df_ml.index.min(), last_ts - pd.Timedelta(hours=HISTORY_HOURS))
recent_truth = df_ml.loc[hist_start:last_ts, "humidity"]

# 4) Plot: historia reciente + predicción futura
fig_future = go.Figure()

fig_future.add_scatter(
    x=recent_truth.index, y=recent_truth.values,
    mode="lines", name="Humedad (real) – reciente"
)

fig_future.add_scatter(
    x=future_index, y=y_future,
    mode="lines+markers", name=f"Predicción próxima {HORIZON} h (CNN)"
)

# Línea vertical separando pasado/futuro
fig_future.add_vline(x=last_ts, line_dash="dot", line_color="gray")

fig_future.update_layout(
    title=f"Nowcasting de humedad — Próximas {HORIZON} horas",
    xaxis_title="Tiempo",
    yaxis_title="Humedad (%)",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig_future.show()


C:\Users\esteb\AppData\Local\Temp\ipykernel_39868\1938096111.py:15: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

